In [ ]:
import sys
sys.path.append('/gpfs/exfel/exp/SPB/202501/p006933/usr/Software/analysistools')
sys.path.append('/gpfs/exfel/exp/SPB/202501/p006933/usr/Software')
import data_helper as dh
from generatorpipeline.generatorpipeline import generatorpipeline as gp
from generatorpipeline.generatorpipeline import accumulators
import extra_data as ex
import numpy as np
import xarray as xr
import dask.array as da

In [ ]:
run=133
data = ex.open_run(6933, run)

xgm_field = dh.det['xgm9']
data[xgm_field].keys()

In [ ]:
%%time
intensity = data[xgm_field, 'data.intensitySa1TD'].xarray()
intensity = data[xgm_field, 'data.intensitySa1TD'].xarray()
filtered_intensity = intensity.where(intensity != 1).dropna(dim='dim_0').isel(dim_0=slice(1,None))
np.shape(intensity)

In [ ]:
def pulse_energy(run, xgm='xgm9'):
    '''
    returns 
    ----------
    a xarray with the pulse energy for the 
    to access one trainId t_id: data.sel(trainId=t_id)
    '''
    data = ex.open_run(6933, run)
    xgm_field = dh.det[xgm]
    intensity = data[xgm_field, 'data.intensitySa1TD'].xarray()
    filtered_intensity = intensity.where(intensity != 1).dropna(dim='dim_0').isel(dim_0=slice(1,None))
    return filtered_intensity

def pulse_energy_train(run, xgm='xgm9', flags=False):
    '''
    returns 
    ----------
    a generator that gives the pulse_energy for trains using pulse_energy(run, xgm='xgm9') and data.sel(trainId=t_id).
    Only exist as an option to directly apply the hitfinderflag when flags=True
    '''
    if flags:
        flag_array=ds[dh.det['hitfinder'], 'data.hitFlag'].xarray()
    pulse_energies=pulse_energy(run, xgm=xgm)
    for t_id in pulse_energies.coords['trainId'].values:
        pulse_energies_train=pulse_energies.sel(trainId=t_id).copy()
        if flags:
            mask = flag_array.sel(trainId=t_id)
            mask=mask.rename({'trainId':'dim_0'})
            yield pulse_energies_train.where(mask).dropna(dim='dim_0')
        else:
            yield pulse_energies_train




In [ ]:
%%time
gen=pulse_energy_train(run, flags=True)


In [ ]:
%%time
test=next(gen)

In [ ]:
test
#plt.plot(test.dropna(dim='dim_0'))

In [ ]:
data[dh.det['hitfinder']].keys()

In [ ]:
%%time
np.shape(filtered_intensity)

In [ ]:
trimmed = filtered_intensity.isel(dim_0=slice(1,None))


In [ ]:
isel(time=slice(1, None))